In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.special import expit
from sklearn.metrics import roc_auc_score

pd.options.mode.chained_assignment = None

In [35]:
def get_var_and_prob(df, path, prot):
    df["var_script"] = None
    df["var_2"] = None
    for i, row in tqdm(enumerate(df.iterrows())):
        _, uniprot_id, pos_c, aa1_c, aa2_c, score_c, pos_a, aa1_a, aa2_a, score_a, score_a_dash, score_delta, score_c_esm, score_a_esm, score_a_dash_esm, var1, var2 = row[1]
        try:
            data1 = pd.read_csv(f'{path}{prot}_{aa1_c}{pos_c}{aa2_c}.diff_mean_var.tsv')
            data2 = pd.read_csv(f'{path}{prot}_{aa1_c}{pos_c}{aa2_c}.diff.llr.tsv')
            data2['new_row'] = data2['row'].apply(lambda x: x.split()[0])
            data2 = data2[data2['column'] != data2['new_row']]

            df.loc[i, "var_script"] = data1.loc[0, 'mean_global']
            df.loc[i, "var_2"] = data2['esm1b_score'].var()
        except:
            ('NO')
    df['var_2'] = pd.to_numeric(df['var_2'], errors='coerce')
    df['prob_esm1b'] = expit(df['var_2'])
    return df

In [36]:
def get_grouped_gt5(df):
    df_grouped = df.groupby(['Pos.C', 'AA1.C', 'AA2.C'], as_index=False).agg({'Protein': 'count', 'Score.Delta': 'var', 'prob_esm1b': 'mean'})
    df_grouped_gt5 = df_grouped[df_grouped.Protein >= 5]
    df_grouped_gt5['prob_dms'] = expit(df_grouped_gt5['Score.Delta'])
    return df_grouped_gt5

In [37]:
ubc9 = pd.read_csv('/Users/xtina/_thesis/res_fin_df/ubc9_all.tsv', sep='\t')
yap1 = pd.read_csv('res_fin_df/yap1_all.tsv', sep='\t')
pabp = pd.read_csv('res_fin_df/pabp_all.tsv', sep='\t')
ube4b = pd.read_csv('res_fin_df/ube4b_all.tsv', sep='\t')
brca1_db6 = pd.read_csv('res_fin_df/brca1_db6_all.tsv', sep='\t')
brca1_db8 = pd.read_csv('res_fin_df/brca1_db8_all.tsv', sep='\t')
spg1 = pd.read_csv('res_fin_df/spg1_all.tsv', sep='\t')

# UBC9

In [38]:
ubc9 = get_var_and_prob(ubc9, 'res_new_w_diff/ubc9/', 'UBC9')

2302it [00:07, 311.11it/s]


In [39]:
ubc9_gr = get_grouped_gt5(ubc9)

/var/folders/q4/tv8rtttx0l5dyw3jknz0v_g00000gn/T/ipykernel_3643/2081782648.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grouped_gt5['prob_dms'] = expit(df_grouped_gt5['Score.Delta'])


In [40]:
ubc9_gr

,Pos.C,AA1.C,AA2.C,Protein,Score.Delta,prob_esm1b,prob_dms
0,2,S,L,11,1.450062,0.813453,0.810008
3,2,S,T,5,0.639249,0.639286,0.654584
6,3,G,R,5,5.786489,0.759778,0.996941
12,4,I,T,5,10.348147,0.627057,0.999968
14,5,A,G,6,0.139721,0.575261,0.534873
...,...,...,...,...,...,...,...
934,151,Q,R,7,8.169454,0.824340,0.999717
945,153,K,R,6,2.006874,0.504024,0.881517
947,154,K,E,5,2.464224,0.692332,0.921595
952,155,F,L,8,2.600668,0.633939,0.930905


# YAP1

In [41]:
yap1 = get_var_and_prob(yap1, 'res_new_w_diff/yap1/', 'YAP1')

38720it [04:05, 157.69it/s]


In [42]:
yap1_gr = get_grouped_gt5(yap1)

/var/folders/q4/tv8rtttx0l5dyw3jknz0v_g00000gn/T/ipykernel_3643/2081782648.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grouped_gt5['prob_dms'] = expit(df_grouped_gt5['Score.Delta'])


In [43]:
yap1_gr

,Pos.C,AA1.C,AA2.C,Protein,Score.Delta,prob_esm1b,prob_dms
0,170,D,A,191,0.232894,0.522800,0.557962
1,170,D,E,217,0.205111,0.501719,0.551099
3,170,D,G,193,0.175458,0.527114,0.543752
4,170,D,H,197,0.212177,0.521694,0.552846
6,170,D,K,11,0.568974,0.557231,0.638526
...,...,...,...,...,...,...,...
373,203,R,H,200,0.971108,0.549894,0.725340
375,203,R,L,190,0.930147,0.615678,0.717105
377,203,R,P,163,1.278060,0.658693,0.782119
379,203,R,S,187,0.903585,0.604053,0.711686


# PABP

In [44]:
pabp = get_var_and_prob(pabp, 'res_new_w_diff/pabp/', 'PABP')
pabp_gr = get_grouped_gt5(pabp)

73022it [08:34, 142.06it/s]
/var/folders/q4/tv8rtttx0l5dyw3jknz0v_g00000gn/T/ipykernel_3643/2081782648.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grouped_gt5['prob_dms'] = expit(df_grouped_gt5['Score.Delta'])


In [45]:
pabp_gr

,Pos.C,AA1.C,AA2.C,Protein,Score.Delta,prob_esm1b,prob_dms
0,126,G,A,143,0.392361,0.507391,0.596851
1,126,G,C,115,0.561419,0.524601,0.636781
2,126,G,D,121,0.797748,0.535677,0.689493
3,126,G,R,89,0.610286,0.541280,0.648006
4,126,G,S,117,0.419936,0.516799,0.603468
...,...,...,...,...,...,...,...
1051,200,P,H,110,0.300182,0.502849,0.574487
1052,200,P,L,114,0.863540,0.505609,0.703400
1053,200,P,R,134,0.149141,0.506164,0.537216
1054,200,P,S,112,0.402614,0.504205,0.599315


# UBE4B

In [46]:
ube4b = get_var_and_prob(ube4b, 'res_new_w_diff/ube4b/', 'UBE4B')
ube4b_gr = get_grouped_gt5(ube4b)

103798it [22:29, 76.91it/s]
/var/folders/q4/tv8rtttx0l5dyw3jknz0v_g00000gn/T/ipykernel_3643/2081782648.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grouped_gt5['prob_dms'] = expit(df_grouped_gt5['Score.Delta'])


In [49]:
ube4b_gr

,Pos.C,AA1.C,AA2.C,Protein,Score.Delta,prob_esm1b,prob_dms
1,1072,I,K,165,0.590486,0.549400,0.643477
2,1072,I,L,364,0.501551,0.509738,0.622824
3,1072,I,M,244,0.600365,0.513502,0.645740
4,1072,I,R,168,0.580610,0.552835,0.641208
6,1072,I,T,151,0.554350,0.522324,0.635144
...,...,...,...,...,...,...,...
789,1173,H,N,193,0.437316,0.500588,0.607619
790,1173,H,P,275,0.426929,0.500729,0.605140
791,1173,H,Q,326,0.515443,0.500503,0.626082
792,1173,H,R,216,0.355197,0.500811,0.587877


# BRCA1_6

In [47]:
brca1_db6 = get_var_and_prob(brca1_db6, 'res_new_w_diff/brca1_6/', 'BRCA1_db6')
brca1_db6_gr = get_grouped_gt5(brca1_db6)

4304it [01:24, 50.82it/s] 
/var/folders/q4/tv8rtttx0l5dyw3jknz0v_g00000gn/T/ipykernel_3643/2081782648.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grouped_gt5['prob_dms'] = expit(df_grouped_gt5['Score.Delta'])


In [48]:
brca1_db6_gr

,Pos.C,AA1.C,AA2.C,Protein,Score.Delta,prob_esm1b,prob_dms
1,2,D,N,11,1.106278,0.501985,0.751435
2,2,D,Y,14,2.240544,0.509837,0.903832
7,4,S,F,7,1.553939,0.505959,0.825482
11,4,S,Y,11,2.469525,0.509356,0.921978
16,5,A,T,15,0.858340,0.500442,0.702314
...,...,...,...,...,...,...,...
1730,299,V,I,15,0.867338,0.500098,0.704191
1734,300,E,K,18,1.552273,0.500259,0.825242
1741,302,A,T,30,1.373682,0.500158,0.797974
1744,303,E,K,14,0.858941,0.500338,0.702439


# BRCA1_8

In [50]:
brca1_db8 = get_var_and_prob(brca1_db8, 'res_new_w_diff/brca1_8/', 'BRCA1_db8')
brca1_db8_gr = get_grouped_gt5(brca1_db8)

8426it [02:48, 50.05it/s] 
/var/folders/q4/tv8rtttx0l5dyw3jknz0v_g00000gn/T/ipykernel_3643/2081782648.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grouped_gt5['prob_dms'] = expit(df_grouped_gt5['Score.Delta'])


# SPG1

In [51]:
spg1 = get_var_and_prob(spg1, 'res_new_w_diff/spg1/', 'SPG1')
spg1_gr = get_grouped_gt5(spg1)

1071834it [1:50:10, 162.15it/s]


In [52]:
spg1_gr

,Pos.C,AA1.C,AA2.C,Protein,Score.Delta,prob_esm1b,prob_dms
0,228,Q,A,1026,0.582932,NaN,0.641742
1,228,Q,C,1025,0.666489,NaN,0.660717
2,228,Q,D,1026,1.203155,NaN,0.769086
3,228,Q,E,1026,0.874267,NaN,0.705633
4,228,Q,F,1024,0.620293,NaN,0.650285
...,...,...,...,...,...,...,...
1040,282,E,S,1026,1.028062,0.519191,0.736540
1041,282,E,T,1026,1.015061,0.515713,0.734009
1042,282,E,V,1026,NaN,0.517706,NaN
1043,282,E,W,1026,NaN,0.541015,NaN


In [55]:
spg1_gr.dropna()

,Pos.C,AA1.C,AA2.C,Protein,Score.Delta,prob_esm1b,prob_dms
19,229,Y,A,1026,4.244752,0.563256,0.985863
20,229,Y,C,1026,4.302655,0.554629,0.986648
21,229,Y,D,1026,9.192321,0.576893,0.999898
22,229,Y,E,1025,9.457020,0.593146,0.999922
23,229,Y,F,1026,0.617142,0.509228,0.649568
...,...,...,...,...,...,...,...
1023,281,T,V,1026,0.590519,0.516998,0.643484
1024,281,T,W,1026,0.588071,0.589304,0.642922
1025,281,T,Y,1026,0.824153,0.539764,0.695117
1040,282,E,S,1026,1.028062,0.519191,0.736540


In [54]:
spg1_gr.to_csv('res_fin_df/proba/sgp1_proba.csv', index=False)
brca1_db8_gr.to_csv('res_fin_df/proba/brca1_db8_proba.csv', index=False)
brca1_db6_gr.to_csv('res_fin_df/proba/brca1_db6_proba.csv', index=False)
ube4b_gr.to_csv('res_fin_df/proba/ube4b_proba.csv', index=False)
pabp_gr.to_csv('res_fin_df/proba/pabp_proba.csv', index=False)
yap1_gr.to_csv('res_fin_df/proba/yap1_proba.csv', index=False)
ubc9_gr.to_csv('res_fin_df/proba/ubc9_proba.csv', index=False)

# ROC-AUC

In [3]:
spg1_gr = pd.read_csv('res_fin_df/proba/sgp1_proba.csv')
brca1_db8_gr = pd.read_csv('res_fin_df/proba/brca1_db8_proba.csv')
brca1_db6_gr = pd.read_csv('res_fin_df/proba/brca1_db6_proba.csv')
ube4b_gr = pd.read_csv('res_fin_df/proba/ube4b_proba.csv')
pabp_gr = pd.read_csv('res_fin_df/proba/pabp_proba.csv')
yap1_gr = pd.read_csv('res_fin_df/proba/yap1_proba.csv')
ubc9_gr = pd.read_csv('res_fin_df/proba/ubc9_proba.csv')

In [11]:
list_of_df = [spg1_gr, brca1_db8_gr, brca1_db6_gr, ube4b_gr, pabp_gr, yap1_gr, ubc9_gr]
list_proteins = ['spg1', 'brca1_db8', 'brca1_db6', 'ube4b', 'pabp', 'yap1', 'ubc9']

proba = pd.DataFrame(columns=list_proteins, index=['roc_auc'])
for df, prot in zip(list_of_df, list_proteins):
    df = df.dropna()
    df['target'] = (df['Score.Delta']> 1.0).astype(int)
    proba.loc['roc_auc', prot] = roc_auc_score(df['target'], df['prob_esm1b'])

proba

,spg1,brca1_db8,brca1_db6,ube4b,pabp,yap1,ubc9
roc_auc,0.688544,0.458595,0.46812,0.844462,0.837289,0.839233,0.69
